In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import *

# Study data load

In [5]:
train_data = pd.read_excel('data/study_data.xlsx')

In [6]:
train_data

,category_seq1,product_seq,product_title
0,17,46231071,레드페이스 남성 반팔티(100)L 13000원
1,13,46231074,카카오프렌즈 교통카드 미개봉 새상품 (무지) 팝니다
2,14,46231062,최신도로공학
3,10,46231064,사당역)욕실장 싸게팔아요
4,16,46231070,60v 10-15a 미니 가변 급충 팝니다.
...,...,...,...
9995,7,46221899,LG 듀얼 인버터 스탠드 에어컨 판매해요
9996,7,46221856,(판매)(전국설치)신품 삼성 업소용 천정형인버터냉난방기 설치/판매!
9997,8,46221860,"[판매]@그래픽카드 정품신품대량,메인보드,램,ssd,파워케이스 대량전문 용산"
9998,11,46221846,네오플램 피카 인덕션 미개봉새상품


In [7]:
len(train_data)

10000

중복 데이터 확인

In [8]:
train_data['product_title'].nunique()

9552

중복 데이터 제거 및 Null 값 제거

In [9]:
train_data.drop_duplicates(subset=['product_title'], inplace=True) # 중복 제거

In [10]:
train_data = train_data.dropna(how='any') # null 값이 존재하는행 제거

In [11]:
print('샘플 데이터 수 : ', len(train_data))

샘플 데이터 수 :  9552


정규 표현식을 이용하여 한글 영어 숫자 _ - + 띄어어쓰기기 남기고 제거

In [14]:
train_data['product_title'] = train_data['product_title'].str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9-_+ ]', '', regex=True)

In [16]:
train_data

,category_seq1,product_seq,product_title
0,17,46231071,레드페이스 남성 반팔티100L 13000원
1,13,46231074,카카오프렌즈 교통카드 미개봉 새상품 무지 팝니다
2,14,46231062,최신도로공학
3,10,46231064,사당역욕실장 싸게팔아요
4,16,46231070,60v 10-15a 미니 가변 급충 팝니다
...,...,...,...
9995,7,46221899,LG 듀얼 인버터 스탠드 에어컨 판매해요
9996,7,46221856,판매전국설치신품 삼성 업소용 천정형인버터냉난방기 설치판매
9997,8,46221860,판매그래픽카드 정품신품대량메인보드램ssd파워케이스 대량전문 용산
9998,11,46221846,네오플램 피카 인덕션 미개봉새상품


empty value -> Null로 변경

In [17]:
train_data['product_title'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

category_seq1    0
product_seq      0
product_title    0
dtype: int64


In [18]:
train_data = train_data.dropna(how = 'any')
print(len(train_data))

9552


불용어 제거 및 형태소 분석

In [19]:
stopwords = ['팝니다', '팔아요', '팜', '판매']

In [20]:
okt = Okt()

In [21]:
x_train = []

In [22]:
for sentence in train_data['product_title']:
    temp = okt.morphs(sentence, stem=True) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    x_train.append(temp)

In [23]:
x_train[:10]

[['레드', '페이스', '남성', '반', '팔', '티', '100', 'L', '13000원'],
 ['카카오', '프렌즈', '교통카드', '미개', '봉', '새', '상품', '무지', '파다'],
 ['최신', '도로', '공학'],
 ['사당역', '욕실', '장', '싸다', '파다'],
 ['60', 'v', '10-15', 'a', '미니', '가변', '급충', '파다'],
 ['부산', '택포', '내셔널', '지오', '그래픽', '투', '웨이', '아웃', '포켓', '백', '팩', '하다'],
 ['매입', '사다', '컴퓨터', '부품', 'CPU', 'RAM', 'VGA', 'HDD', 'SSD', '빠르다', '처리'],
 ['에', '메필', '에블린', '속옷'],
 ['LG', '전자', '시', '네', '빔', '레이저', '4', 'K', 'HU', '80', 'KA'],
 ['삼성', '무풍', '벽', '걸이', '에어컨', '하다']]

integer encoding

In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [25]:
print(tokenizer.word_index)

{'파다': 1, '하다': 2, '새': 3, '갤럭시': 4, 's': 5, '2': 6, '상품': 7, 'g': 8, '블랙': 9, '가': 10, 'a': 11, '-': 12, '+': 13, '사이즈': 14, '급': 15, 'gb': 16, '1': 17, '3': 18, '제품': 19, '256': 20, '폰': 21, '20': 22, '기': 23, '미개': 24, '봉': 25, '5': 26, '4': 27, '128': 28, '정품': 29, '10': 30, '나이키': 31, 'm': 32, '화이트': 33, '백': 34, '용': 35, '에': 36, '권': 37, '노트': 38, '세트': 39, '인치': 40, '반': 41, '삼성': 42, '매입': 43, '자켓': 44, '원피스': 45, 'lg': 46, '택포': 47, '이': 48, 'x': 49, '팔': 50, '상': 51, '7': 52, '8': 53, '개': 54, '중고': 55, '가격': 56, '일괄': 57, '프로': 58, '리퍼': 59, '아이폰': 60, 'l': 61, '100': 62, '스타벅스': 63, '가능': 64, 'k': 65, '플러스': 66, '크다': 67, '티': 68, '계': 69, '지': 70, '64': 71, '공기': 72, '무잔': 73, '6': 74, '배터리': 75, '울트라': 76, '라': 77, '그레이': 78, '9': 79, '블루': 80, '센터': 81, '로': 82, '32': 83, '아': 84, '미니': 85, '포함': 86, '셔츠': 87, '최저': 88, '여성': 89, 'i': 90, '즈': 91, '박스': 92, '장': 93, '판': 94, '제': 95, '21': 96, '고': 97, '스': 98, '남성': 99, '에어': 100, '풀': 101, '대': 102, '네': 103, '세대': 10

In [26]:
print(tokenizer.word_counts)

OrderedDict([('레드', 89), ('페이스', 15), ('남성', 118), ('반', 220), ('팔', 190), ('티', 150), ('100', 162), ('l', 163), ('13000원', 15), ('카카오', 24), ('프렌즈', 17), ('교통카드', 2), ('미개', 312), ('봉', 310), ('새', 909), ('상품', 551), ('무지', 2), ('파다', 1199), ('최신', 19), ('도로', 2), ('공학', 3), ('사당역', 1), ('욕실', 4), ('장', 130), ('싸다', 61), ('60', 24), ('v', 76), ('10-15', 1), ('a', 504), ('미니', 136), ('가변', 1), ('급충', 1), ('부산', 64), ('택포', 191), ('내셔널', 6), ('지오', 7), ('그래픽', 15), ('투', 30), ('웨이', 37), ('아웃', 9), ('포켓', 7), ('백', 243), ('팩', 78), ('하다', 1168), ('매입', 211), ('사다', 75), ('컴퓨터', 68), ('부품', 28), ('cpu', 18), ('ram', 6), ('vga', 6), ('hdd', 8), ('ssd', 26), ('빠르다', 12), ('처리', 3), ('에', 237), ('메필', 2), ('에블린', 1), ('속옷', 7), ('lg', 193), ('전자', 29), ('시', 70), ('네', 114), ('빔', 31), ('레이저', 47), ('4', 297), ('k', 159), ('hu', 2), ('80', 32), ('ka', 2), ('삼성', 214), ('무풍', 3), ('벽', 18), ('걸이', 20), ('에어컨', 40), ('현', 7), ('재판매', 4), ('중', 64), ('55-66', 11), ('사이', 12), ('즈', 131), ('포함'

In [27]:
x_train[:5]

[['레드', '페이스', '남성', '반', '팔', '티', '100', 'L', '13000원'],
 ['카카오', '프렌즈', '교통카드', '미개', '봉', '새', '상품', '무지', '파다'],
 ['최신', '도로', '공학'],
 ['사당역', '욕실', '장', '싸다', '파다'],
 ['60', 'v', '10-15', 'a', '미니', '가변', '급충', '파다']]

In [28]:
encoded = tokenizer.texts_to_sequences(x_train)

In [29]:
encoded[:5]

[[126, 914, 99, 41, 50, 68, 62, 61, 915],
 [557, 808, 4219, 24, 25, 3, 7, 4220, 1],
 [718, 4221, 3238],
 [6150, 2674, 93, 209, 1],
 [558, 156, 6151, 11, 85, 6152, 6153, 1]]

In [30]:
tokenizer.word_index['가격']

56